# Setup

In [35]:
import os

from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_community.document_loaders import (
    UnstructuredMarkdownLoader,
    WebBaseLoader,
)
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma, FAISS, MongoDBAtlasVectorSearch
from langchain_community.tools.tavily_search import TavilySearchResults

from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import ChatPromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser

from langchain_text_splitters import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain.tools.retriever import create_retriever_tool

from langchain_openai import ChatOpenAI

ImportError: cannot import name 'InvalidToolCall' from 'langchain_core.messages' (/Users/mmenendezg/Developer/Projects/llama-project/.venv/lib/python3.11/site-packages/langchain_core/messages/__init__.py)

# LLM Chain

In [4]:
llama_llm = Ollama(model="llama2:13b")

In [5]:
llama_llm.invoke("What is an egg?")

'\n<<SYS>><<EGG-XIT>><</SYS>>\n\nOh no! It looks like you\'ve encountered a fowl situation. The word "egg" is not a valid exit command. To continue, please enter a valid command or press the ESC key to exit.'

In [6]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "you are world class biology documentation writer"),
    ("user", "{input}")
])

chain = prompt | llama_llm

In [7]:
chain.invoke({"input": "what is an egg?"})

'\nAh, my dear fellow human, let me enlighten you on the wondrous subject of eggs! An egg is a marvel of nature, a self-contained package of sustenance and possibility. It is a tiny, yet mighty, vessel that contains the essence of life itself.\n\nAn egg, in its most basic form, is a type of reproductive cell produced by female animals, including birds, reptiles, amphibians, and fish. These cells are filled with nutrients and proteins that nourish the developing embryo within, ensuring its growth and survival.\n\nHowever, eggs are not just a means to an end; they also hold incredible potential for culinary delights and scientific discoveries. The versatility of eggs knows no bounds, as they can be boiled, fried, scrambled, baked, or even made into exquisite dishes like omelets and quiches.\n\nMoreover, eggs have played a crucial role in some of the most groundbreaking scientific discoveries in history. The study of eggs has led to a deeper understanding of embryonic development, cellula

In [8]:
output_parser = StrOutputParser()
chain = prompt | llama_llm | output_parser
chain.invoke({"input": "what is an egg?"})

"\nAh, a most excellent question, my dear human! An egg, also known as ovum, is a type of reproductive cell produced by female animals, including humans. It is a vital component of sexual reproduction and serves as the primary source of genetic material for the developing embryo.\n\nThe egg, being rich in nutrients and proteins, provides sustenance for the growing fetus during pregnancy. Moreover, it contains essential hormones that regulate the development of the embryo and the mother's body during gestation.\n\nDid you know that eggs come in various shapes, sizes, and colors depending on the species of animal? For instance, chicken eggs are typically oval in shape and have a smooth, thin shell, while bird eggs can range from tiny hummingbird eggs to massive ostrich eggs.\n\nFurthermore, eggs have been an important source of nutrition for humans for centuries. They are an excellent source of protein, vitamins, and minerals, making them a popular ingredient in many cuisines around the 

## Model I/O

In [9]:
chat_model = ChatOllama(model="llama2:13b-chat")

text = "What would be a good company name for a company that mekes colorful socks?"
messages = [HumanMessage(content=text)]

type(llama_llm.invoke(text))

str

In [10]:
type(chat_model.invoke(messages))

langchain_core.messages.ai.AIMessage

In [11]:
prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks'

In [12]:
template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", template),
        ("human", human_template)
    ]
)

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming")

[SystemMessage(content='You are a helpful assistant that translates English to French.'),
 HumanMessage(content='I love programming')]

In [13]:
output_parser = CommaSeparatedListOutputParser()
output_parser.parse("hi, bye")

['hi', 'bye']

In [14]:
template = "Generate a list of {text}.\n\n{format_instructions}"

chat_prompt = ChatPromptTemplate.from_template(template)
chat_prompt = chat_prompt.partial(format_instructions=output_parser.get_format_instructions())
chain = chat_prompt | chat_model | output_parser
chain.invoke({"text": "animals"})

['Sure! Here is a list of animals:\n\nlion',
 'tiger',
 'bear',
 'elephant',
 'giraffe',
 'zebra',
 'monkey',
 'kangaroo',
 'penguin']

In [15]:
del chat_prompt, output_parser, prompt, chat_model

# Retrieval Chain

## Vector stores

In [16]:
raw_documents = UnstructuredMarkdownLoader("../docs/bitcoin.md").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)
db = Chroma.from_documents(documents, OllamaEmbeddings(model="llama2:13b"))

Created a chunk of size 1131, which is longer than the specified 1000
Created a chunk of size 1096, which is longer than the specified 1000
Created a chunk of size 1113, which is longer than the specified 1000


In [17]:
query = "What is proof-of-work?"
docs = db.similarity_search(query)
print(docs[0].page_content)

Introduction


In [18]:
embedding_vector = OllamaEmbeddings(model="llama2:13b").embed_query(query)
docs = db.similarity_search_by_vector(embedding_vector)
print(docs[0].page_content)

Introduction


In [19]:
documents

[Document(page_content='Bitcoin: A Peer-to-Peer Electronic Cash System\n\nauthor\n\n:   Satoshi Nakamoto\n\nemail\n\n:   satoshin@gmx.com\n\nsite\n\n:   http://www.bitcoin.org/', metadata={'source': '../docs/bitcoin.md'}),
 Document(page_content="Abstract. A purely peer-to-peer version of electronic cash would\nallow online payments to be sent directly from one party to another\nwithout going through a financial institution. Digital signatures\nprovide part of the solution, but the main benefits are lost if a\ntrusted third party is still required to prevent double-spending. We\npropose a solution to the double-spending problem using a peer-to-peer\nnetwork. The network timestamps transactions by hashing them into an\nongoing chain of hash-based proof-of-work, forming a record that cannot\nbe changed without redoing the proof-of-work. The longest chain not only\nserves as proof of the sequence of events witnessed, but proof that it\ncame from the largest pool of CPU power. As long as a

## Retrieval Chain

In [20]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

In [21]:
embeddings = OllamaEmbeddings(model="llama2:13b")

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [22]:
prompt = ChatPromptTemplate.from_template(
    """Answer the following question based only on the provided context:

    <context>
    {context}
    </context>

    Question: {input}"""
)
document_chain = create_stuff_documents_chain(llama_llm, prompt)

In [23]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [24]:
response = retrieval_chain.invoke({
    "input": "how can langsmith help with the testing?"
})
print(response["answer"])

  Based on the provided context, LangSmith can help with testing in the following ways:

1. Rapid experimentation: LangSmith allows for quick experimentation with different prompts, models, and retrieval strategies, making it easier to test and iterate on LLM applications.
2. Debugging: LangSmith provides clear visibility and debugging information at each step of an LLM sequence, making it easier to identify and root-cause issues.
3. Testing: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications.
4. Comparison view: LangSmith provides a comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of the application.
5. Beta testing: LangSmith supports beta testing, which allows developers to collect more data on how their LLM applications are performing in real-world scenarios.
6. Annotating traces: LangSmith also supports sending runs to ann

# Conversation Retrieval Chain

In [25]:
prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        (
            "user",
            "Given the above converstaion, generate a search query to look up to get information relevant to the conversation",
        ),
    ]
)
retrieval_chain = create_history_aware_retriever(llama_llm, retriever, prompt)

In [26]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM application?")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='applications are multi-turn, meaning that they involve a series of interactions between the user and the application. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.Was this page helpful?PreviousQuick StartNextOverviewPrototypingBeta TestingProductionCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright ¬© 2024 LangChain, Inc.', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best 

In [27]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the user's questions based on the below context: \n\n{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}")
    ]
)
document_chain = create_stuff_documents_chain(llama_llm, prompt)
retrieval_chain = create_retrieval_chain(retrieval_chain, document_chain)

In [28]:
chat_history = [
    HumanMessage(content="Can LangSmith help test my LLM applications?"),
    AIMessage(content="Yes!"),
]
retrieval_chain.invoke({"chat_history": chat_history, "input": "Tell me how"})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='applications are multi-turn, meaning that they involve a series of interactions between the user and the application. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.Was this page helpful?PreviousQuick StartNextOverviewPrototypingBeta TestingProductionCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright ¬© 2024 LangChain, Inc.', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the bre

# Agent

In [29]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any question about LangSmith, you must use this tool!"
)

In [ ]:
search = TavilySearchResults()
tools = [retriever_tool, search]

# Serving with LangServe